In [15]:
import json
import logging
import math
import os
import random
import re
import shutil
import sys
from collections import defaultdict
from pathlib import Path

import numpy as np
import pandas as pd
from icecream import ic
from pandas import DataFrame
from tqdm import tqdm
import transformers


def load_tokenizer():
    """  
    Currently, used special tokens are `<|endoftext|>` in Qwen-7B, and `<|endoftext|>`, 
    `<|im_start|>`, and `<|im_end|>` in Qwen-7B-Chat,    
    """
    cache_dir = None
    model_max_length = 512
    model_name_or_path = '/mnt/nas1/models/qwen/Qwen-7B-Chat-Int8'
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_name_or_path,
        cache_dir=cache_dir,
        model_max_length=model_max_length,
        padding_side="right",
        use_fast=False,
        trust_remote_code=True,
    )
    tokenizer.pad_token_id = tokenizer.eod_id
    return tokenizer


tokenizer = load_tokenizer()
end_token = "<|endoftext|>"
im_start = tokenizer.im_start_id
im_end = tokenizer.im_end_id
eod_id = tokenizer.eod_id
roles = {"user": "<|im_start|>user", "assistant": "<|im_start|>assistant"}
existent_token_file = '/mnt/nas1/models/qwen/Qwen-7B-Chat-Int8/qwen.tiktoken'
print(tokenizer.special_tokens)
print(im_start, im_end, eod_id)
with open(existent_token_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
print(len(lines), lines[0], lines[-1])
# tokenizer convert to bytes level, not char level
# word 一只 is already a token b'\xe4\xb8\x80\xe5\x8f\xaa', skipping
# for line in lines:
#     if '\xe4\xb8\x80\xe5\x8f\xaa' in line:
#         print(line)
#         break

{'<|endoftext|>': 151643, '<|im_start|>': 151644, '<|im_end|>': 151645, '<|extra_0|>': 151646, '<|extra_1|>': 151647, '<|extra_2|>': 151648, '<|extra_3|>': 151649, '<|extra_4|>': 151650, '<|extra_5|>': 151651, '<|extra_6|>': 151652, '<|extra_7|>': 151653, '<|extra_8|>': 151654, '<|extra_9|>': 151655, '<|extra_10|>': 151656, '<|extra_11|>': 151657, '<|extra_12|>': 151658, '<|extra_13|>': 151659, '<|extra_14|>': 151660, '<|extra_15|>': 151661, '<|extra_16|>': 151662, '<|extra_17|>': 151663, '<|extra_18|>': 151664, '<|extra_19|>': 151665, '<|extra_20|>': 151666, '<|extra_21|>': 151667, '<|extra_22|>': 151668, '<|extra_23|>': 151669, '<|extra_24|>': 151670, '<|extra_25|>': 151671, '<|extra_26|>': 151672, '<|extra_27|>': 151673, '<|extra_28|>': 151674, '<|extra_29|>': 151675, '<|extra_30|>': 151676, '<|extra_31|>': 151677, '<|extra_32|>': 151678, '<|extra_33|>': 151679, '<|extra_34|>': 151680, '<|extra_35|>': 151681, '<|extra_36|>': 151682, '<|extra_37|>': 151683, '<|extra_38|>': 151684, '<

In [12]:
b'\xe4\xb8\x80\xe5\x8f\xaa'.decode('utf-8')

'一只'

In [9]:
print(tokenizer.encode("hello world"))
print(tokenizer('print("<|endoftext|>")', allowed_special=set()))
print(tokenizer('print("<|extra_0|>")<|endoftext|>', allowed_special={'<|endoftext|>'}))

[14990, 1879]
{'input_ids': [1350, 9639, 91, 8691, 723, 427, 91, 82598], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [1350, 9639, 91, 15460, 62, 15, 91, 82598, 151643], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
